## Text Analysis
In this lab, you will create a classification model that performs sentiment analysis of tweets.
### Import Spark SQL and Spark ML Libraries

First, import the libraries you will need:

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *

from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer, StopWordsRemover

### Load Source Data
Now load the tweets data into a DataFrame. This data consists of tweets that have been previously captured and classified as positive or negative.

In [4]:
tweets_csv = spark.read.csv('wasb://spark@dbshepstor.blob.core.windows.net/data/tweets.csv', inferSchema=True, header=True)
tweets_csv.show(truncate = False)

+------+---------+---------------+-----------------------------------------------------------+
ItemID|Sentiment|SentimentSource|SentimentText |
+------+---------+---------------+-----------------------------------------------------------+
1038 |1 |Sentiment140 |that film is fantastic #brilliant |
1804 |1 |Sentiment140 |this music is really bad #myband |
1693 |0 |Sentiment140 |winter is terrible #thumbs-down |
1477 |0 |Sentiment140 |this game is awful #nightmare |
45 |1 |Sentiment140 |I love jam #loveit |
246 |0 |Sentiment140 |I dislike skiing #rubbish |
776 |1 |Sentiment140 |I like pop music #toptastic |
1666 |1 |Sentiment140 |this game is awful good |
1237 |0 |Sentiment140 |rock music is terrible #worstever |
1386 |1 |Sentiment140 |that movie is great #favorite |
695 |0 |Sentiment140 |I hate this game #fail |
649 |0 |Sentiment140 |I dislike this game #thumbs-down |
1565 |1 |Sentiment140 |that movie is great #thumbs-up |
150 |1 |Sentiment140 |I like tea #brilliant |
1500 |1 |Sentiment140 |this game is terrible for fans of the other team #nightmare|
1781 |0 |Sentiment140 |this game is terrible #fail |
884 |1 |Sentiment140 |this game is brilliant #loveit |
1793 |0 |Sentiment140 |jam is terrible #fail |
1782 |0 |Sentiment140 |coffee is terrible #fail |
921 |0 |Sentiment140 |that movie is awful #hateit |
+------+---------+---------------+-----------------------------------------------------------+
only showing top 20 rows

### Prepare the Data
The features for the classification model will be derived from the tweet text. The label is the sentiment (1 for positive, 0 for negative)

In [6]:
data = tweets_csv.select("SentimentText", tweets_csv.Sentiment.cast("int").alias("label"))
data.show(truncate = False)

+-----------------------------------------------------------+-----+
SentimentText |label|
+-----------------------------------------------------------+-----+
that film is fantastic #brilliant |1 |
this music is really bad #myband |1 |
winter is terrible #thumbs-down |0 |
this game is awful #nightmare |0 |
I love jam #loveit |1 |
I dislike skiing #rubbish |0 |
I like pop music #toptastic |1 |
this game is awful good |1 |
rock music is terrible #worstever |0 |
that movie is great #favorite |1 |
I hate this game #fail |0 |
I dislike this game #thumbs-down |0 |
that movie is great #thumbs-up |1 |
I like tea #brilliant |1 |
this game is terrible for fans of the other team #nightmare|1 |
this game is terrible #fail |0 |
this game is brilliant #loveit |1 |
jam is terrible #fail |0 |
coffee is terrible #fail |0 |
that movie is awful #hateit |0 |
+-----------------------------------------------------------+-----+
only showing top 20 rows

### Split the Data
In common with most classification modeling processes, you'll split the data into a set for training, and a set for testing the trained model.

In [8]:
splits = data.randomSplit([0.7, 0.3])
train = splits[0]
test = splits[1].withColumnRenamed("label", "trueLabel")
train_rows = train.count()
test_rows = test.count()
print ("Training Rows:", train_rows, " Testing Rows:", test_rows)

Training Rows: 1348 Testing Rows: 584

### Define the Pipeline
The pipeline for the model consist of the following stages:
- A Tokenizer to split the tweets into individual words.
- A StopWordsRemover to remove common words such as "a" or "the" that have little predictive value.
- A HashingTF class to generate numeric vectors from the text values.
- A LogisticRegression algorithm to train a binary classification model.

In [10]:
tokenizer = Tokenizer(inputCol="SentimentText", outputCol="SentimentWords")
swr = StopWordsRemover(inputCol=tokenizer.getOutputCol(), outputCol="MeaningfulWords")
hashTF = HashingTF(inputCol=swr.getOutputCol(), outputCol="features")
lr = LogisticRegression(labelCol="label", featuresCol="features", maxIter=10, regParam=0.01)
pipeline = Pipeline(stages=[tokenizer, swr, hashTF, lr])

### Run the Pipeline as an Estimator
The pipeline itself is an estimator, and so it has a **fit** method that you can call to run the pipeline on a specified DataFrame. In this case, you will run the pipeline on the training data to train a model.

In [12]:
piplineModel = pipeline.fit(train)
print ("Pipeline complete!")

Pipeline complete!

### Test the Pipeline Model
The model produced by the pipeline is a transformer that will apply all of the stages in the pipeline to a specified DataFrame and apply the trained model to generate predictions. In this case, you will transform the **test** DataFrame using the pipeline to generate label predictions.

In [14]:
prediction = piplineModel.transform(test)
predicted = prediction.select("SentimentText", "prediction", "trueLabel")
predicted.show(100, truncate = False)

+------------------------------------+----------+---------+
SentimentText |prediction|trueLabel|
+------------------------------------+----------+---------+
I adore cheese #bestever |1.0 |1 |
I adore cheese #toptastic |1.0 |1 |
I adore classical music #toptastic |1.0 |1 |
I adore coffee #favorite |1.0 |1 |
I adore coffee #loveit |1.0 |1 |
I adore jam #bestever |1.0 |1 |
I adore jam #thumbs-up |1.0 |1 |
I adore pop music #thumbs-up |1.0 |1 |
I adore rock music #favorite |1.0 |1 |
I adore skiing #brilliant |1.0 |1 |
I adore skiing #toptastic |1.0 |1 |
I adore summer #brilliant |1.0 |1 |
I adore summer #favorite |1.0 |1 |
I adore tea #thumbs-up |1.0 |1 |
I adore that band #favorite |1.0 |1 |
I adore that band #toptastic |1.0 |1 |
I adore that film #toptastic |1.0 |1 |
I adore that movie #brilliant |1.0 |1 |
I adore that movie #favorite |1.0 |1 |
I adore that movie #thumbs-up |1.0 |1 |
I adore that movie #toptastic |1.0 |0 |
I adore the holidays #loveit |1.0 |1 |
I adore this band #favorite |1.0 |1 |
I adore this band #thumbs-up |1.0 |1 |
I adore this band #toptastic |1.0 |1 |
I adore this book #bestever |1.0 |1 |
I adore this book #loveit |1.0 |1 |
I adore this book #toptastic |1.0 |1 |
I adore this game #loveit |1.0 |1 |
I adore this game #thumbs-up |1.0 |1 |
I adore this game #toptastic |1.0 |1 |
I adore this team #bestever |1.0 |1 |
I adore this team #loveit |1.0 |1 |
I adore this team #thumbs-up |1.0 |1 |
I adore winter #loveit |1.0 |1 |
I adore winter #toptastic |1.0 |1 |
I dislike cheese #fail |0.0 |0 |
I dislike cheese #rubbish |0.0 |0 |
I dislike cheese #thumbs-down |0.0 |0 |
I dislike classical music #nightmare|0.0 |0 |
I dislike classical music #worstever|0.0 |0 |
I dislike jam #fail |0.0 |0 |
I dislike jam #hateit |0.0 |0 |
I dislike pop music #hateit |0.0 |0 |
I dislike pop music #nightmare |0.0 |0 |
I dislike rock music #rubbish |0.0 |0 |
I dislike skiing #fail |0.0 |0 |
I dislike summer #fail |0.0 |0 |
I dislike tea #fail |0.0 |0 |
I dislike tea #rubbish |0.0 |0 |
I dislike tea #thumbs-down |0.0 |0 |
I dislike tea #worstever |0.0 |0 |
I dislike that band #hateit |0.0 |0 |
I dislike that film #hateit |0.0 |0 |
I dislike that film #rubbish |0.0 |0 |
I dislike that film #worstever |0.0 |0 |
I dislike that movie #fail |0.0 |0 |
I dislike that movie #worstever |0.0 |0 |
I dislike the holidays #hateit |0.0 |0 |
I dislike the holidays #thumbs-down |0.0 |0 |
I dislike the holidays #worstever |0.0 |0 |
I dislike this band #nightmare |0.0 |0 |
I dislike this band #rubbish |0.0 |0 |
I dislike this book #worstever |0.0 |0 |
I dislike this game #fail |0.0 |0 |
I dislike winter #fail |0.0 |0 |
I don't like this game |1.0 |0 |
I hate cheese #fail |0.0 |0 |
I hate cheese #worstever |0.0 |0 |
I hate classical music #hateit |0.0 |0 |
I hate classical music #thumbs-down |0.0 |0 |
I hate coffee #fail |0.0 |0 |
I hate jam #rubbish |0.0 |0 |
I hate pop music #hateit |0.0 |0 |
I hate rock music #fail |0.0 |0 |
I hate skiing #hateit |0.0 |0 |
I hate skiing #nightmare |0.0 |0 |
I hate skiing #thumbs-down |0.0 |0 |
I hate skiing #worstever |0.0 |0 |
I hate summer #nightmare |0.0 |0 |
I hate tea #nightmare |0.0 |0 |
I hate tea #rubbish |0.0 |0 |
I hate tea #worstever |0.0 |0 |
I hate that band #worstever |0.0 |0 |
I hate that film #hateit |0.0 |0 |
I hate that film #rubbish |0.0 |0 |
I hate that film #worstever |0.0 |0 |
I hate that movie #hateit |0.0 |0 |
I hate the holidays #hateit |0.0 |0 |
I hate the holidays #nightmare |0.0 |0 |
I hate the holidays #worstever |0.0 |0 |
I hate this band #fail |0.0 |0 |
I hate this band #worstever |0.0 |0 |
I hate this book #nightmare |0.0 |0 |
I hate this book #rubbish |0.0 |0 |
I hate this book #thumbs-down |0.0 |0 |
I hate this game #hateit |0.0 |0 |
I hate this game #thumbs-down |0.0 |0 |
I hate this team #worstever |0.0 |0 |
I hate winter #hateit |0.0 |0 |
+------------------------------------+----------+---------+
only showing top 100 rows